# top

In [ ]:
%matplotlib widget
import forum_utils
from importlib import reload
reload(forum_utils)

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# 09 April 2023
# nrobot
# Run the acornym expansion on the full dataset!

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import gensim
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_short, remove_stopwords, strip_multiple_whitespaces, strip_non_alphanum
from nltk.corpus import stopwords

import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import Word2Vec

from nltk import PorterStemmer

In [ ]:
columns_page_info = ['time_downloaded', 'author', 'posted_date_readable',  'post_ordinal', 'thread_page_name', 'thread_page_num', 'thread_page_url', 'post_text']

columns_thread_info = ['src_category_name', 'thread_page_name', 'thread_page_num', 'thread_max_pages', 'thread_page_url']

columns_likes = ['num_likers', 'likers']
columns_quotes = ['num_quotes', 'quoted_post_ids', 'quoted_authors', 'quoted_contents']
columns_authors = ['author', 'author_title', 'author_num_posts', 'author_num_reviews', 'author_url', 'join_date_readable', 'join_date_data']

In [ ]:
def load_data(infile='list_of_post_contents.csv', nrows=None):
    infile='list_of_post_contents.csv'

    df = pd.read_csv(Path(os.getcwd(), 'nogit_data', infile), nrows=nrows)
    print(f'{df.columns=}')
    print(f'{df.shape=}')

    df.dropna(subset=['post_text'], inplace=True)
    print(f'{df.shape=}')
    df.drop_duplicates(subset=['post_text'], inplace=True)
    print(f'{df.shape=}')

    print(f'{columns_page_info=}')
    df = df[columns_page_info]
    df['posted_date_datetime'] = df.posted_date_readable.parallel_apply(
        lambda x: pd.to_datetime(x))
    return df

def get_discussions_only(df):
    # remove posts that come from reviews (vs. discussions)
    discussions = df[df.src_category_name.str.contains('Discussion')]
    # reformat 1,000 to 1000
    if discussions.author_num_posts.dtype != int:
        discussions.author_num_posts = discussions.author_num_posts.apply(lambda x: x.replace(',', ''))
        discussions.author_num_posts = discussions.author_num_posts.astype(int)
    return discussions

# preprocess

In [ ]:
CUSTOM_FILTERS = [
    strip_tags, strip_punctuation, strip_multiple_whitespaces, 
    strip_non_alphanum,
    lambda y: remove_stopwords(y, stopwords=my_stopwords),
    lambda z: porter.stem(z, to_lowercase=False ),
    lambda x: strip_short(x, minsize=2),  # remove only 1 letter words 
    ]
def nltk_preprocess(df, filters=None):
    if not filters:
        CUSTOM_FILTERS = [
            strip_tags, strip_punctuation, strip_multiple_whitespaces, 
            strip_non_alphanum,
            lambda y: remove_stopwords(y, stopwords=my_stopwords),
            lambda z: porter.stem(z, to_lowercase=False ),
            lambda x: strip_short(x, minsize=2),  # remove only 1 letter words 
            ]
    else:
        CUSTOM_FILTERS = filters

    my_stopwords = stopwords.words('english')
    print(my_stopwords)
    my_stopwords.extend([s.title() for s in my_stopwords])
    print(f'{my_stopwords=}')
    print(f'{df.columns=}')
    porter = PorterStemmer()


    df['preprocessed_posts'] = df['post_text'].parallel_apply(
        lambda x: preprocess_string(x, CUSTOM_FILTERS)) 
    return df

    #stop_nltk.extend([s.title() for s in stop_nltk])

In [ ]:
my_stopwords = stopwords.words('english')
my_stopwords.extend([s.title() for s in my_stopwords])
porter = PorterStemmer()
#snowball = SnowballStemmer()

preprocess_string('“Contact, us” t to To I\'m I"m marry married marriage Marrying Marriage Married Marry I’ve', CUSTOM_FILTERS)

In [ ]:
def create_bigram_corpus(df, min_count=2, threshold=10): # TODO: consider taking in a phraser model directly, instead of params to pass to one
    posts = df.preprocessed_posts.to_list()
    my_phrases = gensim.models.Phrases(posts, min_count=2, threshold=threshold)
    bigram_ifier = Phraser(my_phrases)

    df['bigrammed_posts'] = df['preprocessed_posts'].parallel_apply(
        lambda post: bigram_ifier[post]) 

    bigrammed_corpus = df.bigrammed_posts.to_list()
    print(f'Created word vectors for corpus size {len(bigrammed_corpus)=}, '
          f'example post {bigrammed_corpus[0]=}')
    return bigrammed_corpus
    
def create_word2vec(corpus, overwrite=False, outfile='nogit_data/Case_1/word2vec.bigrams.model', seed=None):
    model = Word2Vec(corpus, seed=seed, workers=10, vector_size=100, window=5, min_count=1)

    if overwrite:
        # Open "path" for writing, creating any parent directories as needed.
        # TODO: catch exception where parent folder doesn't exist; or 
        # force it to exist by committing to git (normally nogit_data ignored by git)
        # TODO os.makedirs(os.path.dirname(outfile), exist_ok=True)
        outfile = Path(outfile)
        if seed:
            # add in seed to filename
            path, file, ext = outfile.parent, outfile.stem, outfile.suffix
            outfile = Path(path, f'{file}_seed-{seed}{ext}')
        print(f'\-- {overwrite=}, saved model to {outfile=}')
        model.save(str(outfile))
    return model

In [ ]:
%%time 
df = load_data()#nrows=100000)

In [ ]:
DATA_PATH_CASE_2 = Path('nogit_data', 'Case_2')
tag = ''
df = pd.read_pickle(Path(DATA_PATH_CASE_2,f'{tag}preprocessed_df.pd-1.5.1.pkl'))

In [ ]:
%%time
df
#df = load_data(nrows=1000)
#df = load_data(nrows=10000)
#df = load_data(None)d


In [ ]:
%%time 
df = nltk_preprocess(df)
display(df.preprocessed_posts)

In [ ]:
df

In [ ]:
!mkdir nogit_data/Case_1

In [ ]:
%%time 
df.to_pickle(f'nogit_data/Case_1/df.pd_{pd.__version__}.pkl')

In [ ]:
%%time 
bigrammed_corpus = create_bigram_corpus(df)

In [ ]:
df.columns

# reload data

In [ ]:
DATA_PATH = Path('nogit_data', 'Case_2') # it's created by acornmy file
tag = ''
model_bigram = Word2Vec.load(str(Path(DATA_PATH, f'{tag}bigram_word2vec.gensim-4.3.1.model')))
print('loaded word2vec model')

In [ ]:
#import matplotlib
#matplotlib.use('module://ipympl.backend_nbagg')
#%matplotlib ipympl

# explore consisitency across seeds


## by training additional models

In [ ]:
import pathlib

In [ ]:
%%time
#seeds = [1,42,100,12345,888]
seeds = [42]
models = []

for seed in seeds:
    print(f'{"-"* 40}')
    print(f'{seed=}\n')
    
    model = create_word2vec(df.preprocessed_posts, overwrite=False, seed=seed)
    models.append(model)

model_unigram = model

In [ ]:
model_unigram.save(str(pathlib.Path('nogit_data', 'Case_1', f'word2vec_unigram_seed-{seed}.model')))
model_unigram.wv.key_to_index

In [ ]:
model_unigram.wv.most_similar('im')

In [ ]:
# for non brigram 
results = {}

for idx in range(5):
    model = models[idx]
    words = ['worries', 'worrying', 'guilty',  'family', 'Wife', 'SO','partner', 'married',  'LEO', 'law', 'police', 'trafficking', 'arrest']
    for query in words:
        sims = model.wv.most_similar(query, topn=10)  # get other similar words
        #print(f'{query=}\t ' , ', '.join([ f'{word} = {vector:.2f}' for word, vector in sims]))
        results[query] = [ f'{word} = {vector:.2f}' for word, vector in sims]
        #print('-')
    
    display(pd.DataFrame(results))

In [ ]:
model = models[3]
for query in ['paper', 'laptop', 'plant']:#, 'MMS', 'HJ', 'HE', 'BJ', 'full_menu']:
    sims = model.wv.most_similar(query, topn=30)  # get other similar words
    print(f'{query=}\t ' , ', '.join([ f'{word} = {vector:.2f}' for word, vector in sims]))
    print('-')

In [ ]:
----> model seed 42 --->
         word 1    word 2
word 1   top    resuls    
word 2   second similiar

so... that's ....
for each model... have the word
{model: word1: []
 
or .... 
 
 result 1 - which word - which model
 

In [ ]:
i
...

# plot umap

## note: this section abandoned because i would need to subsample the data to plot the umap again with the hovered words

In [ ]:
# !pip install umap-learn # NOT pip install umap

In [ ]:
import umap.umap_ as umap

In [ ]:
negatives = ['anxiety', 'worry']
nonconcern_seeds = ['parking', 'table']
concern_seeds = ['LEO', 'STD']
hypothesis_seeds = ['SO', 'marriage']

In [ ]:
negatives + nonconcern_seeds

In [ ]:
model_unigram.wv.vectors

In [ ]:
umap_results =  umap.UMAP().fit(model_unigram.wv.vectors)

In [ ]:
size_model = len(model_unigram.wv.key_to_index)
size_model

In [ ]:
import umap.plot as uplot

In [ ]:
random_ints = np.round(rng.random(1000) * size_model).astype(int)
random_ints

In [ ]:
import umap.plot
words = model.wv.key_to_index
#words

In [ ]:
model.wv.vectors

### failed attempt to plot umap embeddings with hover text

In [ ]:
flipped_words = {v: k for k, v in words.items()}
umap.plot.output_notebook()
p = umap.plot.interactive(umap_results, hover_data=flipped_words, point_size=2)
umap.plot.show(p)

### non interactive plot

In [ ]:
import umap.umap_ as umap
import matplotlib

# 2 MAy 2023 final plot for paper 

In [ ]:
rng = np.random.default_rng(42)

embedding_clusters = []
word_clusters = []
keys = negatives + nonconcern_seeds + concern_seeds +hypothesis_seeds + ['MMS', 'provider']
# NUMBER IS 10
colors = ['red', 'red', 'gold', 'gold', 'gray', 'gray', 'purple', 'purple', 'darkgreen', 'darkgreen']
colors = ['red', 'red', 'goldenrod', 'goldenrod', 'darkslateblue', 'darkslateblue', 'purple', 'purple', 'forestgreen', 'forestgreen']
#colors = ['red', 'blue', 'gold', 'purple', 'pink', 'gray', 'teal', 'orange', 'hotpink']

SEED = 42
NUM_WORDS = 4
UMAP_N =10 

for word in keys:
    print(f"Key = {word}")
    embeddings = [model_unigram.wv[word]]
    words = [word]
    for similar_word, _ in model_unigram.wv.most_similar(word, topn=NUM_WORDS):
        words.append(similar_word)
        embeddings.append(model_unigram.wv[similar_word])
    embedding_clusters.append(embeddings)
    word_clusters.append(words)
    
embedding_clusters = np.array(embedding_clusters)
n, m, k = embedding_clusters.shape
print(n, m, k)
#tsne_model_unigram_en_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=32, n_jobs=-1)

# looks petty random with n_neighbors=20, looks nice at 67 8
umap_2d = umap.UMAP(n_neighbors=UMAP_N, n_components=2, init='spectral', random_state=SEED)
# his means that low values of n_neighbors will force UMAP to concentrate on very local structure (potentially to the detriment of the big picture), while large values will push UMAP to look at larger neighborhoods of each point wh, default 10

#umap_3d = umap.UMAP(n_neighbors=UMAP_N, n_components=3, init='spectral', random_state=SEED)
embeddings_en_2d = np.array(umap_2d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 2)
#embeddings_en_3d = np.array(umap_3d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 3)

def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

# hack - why are embeddings a bit different ... 
def umap_plot_similar_words(title, labels, embedding_clusters, word_clusters, a, filename=None):
    for _sentiment in negatives:
        figsize = (9.5,6) if (matplotlib.get_backend() == 'nbAgg') else (8,7)  # interactive plot should be smaller
        #figsize = (9.5,6)
        fig = plt.figure(figsize=(figsize))
        #ax = fig.add_subplot(projection='3d')
        plotted_words = []
        for label, embeddings, words, color in zip(labels, embedding_clusters, word_clusters, colors):
            #x = embeddings[:, 0]
            #y = embeddings[:, 1]
            #plt.scatter(x, y, c=[color], alpha=a, label=label, edgecolors='gray', s=50)
            for i, word in enumerate(words):
                if word not in plotted_words:
                    x = embeddings[i, 0]
                    y = embeddings[i, 1]
                    plt.scatter(x, y, c=[color], alpha=a, label=label, edgecolors='gray')
                    if word =='worry':
                        print(f'{x=}, {y=}')
                    sign = -1 if np.random.random() > 0.5 else 1
                    if word in keys:
                        dist = model_unigram.wv.similarity(_sentiment, word)
                        plt.annotate(f'{word}, {dist:.2f}', alpha=.8, xy=(x, y), xytext=(0,0),#(2 + rng.random()*4, sign*(4 + rng.random()*4)),
                                    textcoords='offset points', ha='right', va='bottom', size=10,  weight='bold',)
                    else:
                        plt.annotate(f'{word}', alpha=.8, xy=(x, y), xytext=(0,0),#(2 + rng.random()*4, sign*(4 + rng.random()*4)),
                                    textcoords='offset points', ha='right', va='bottom', size=10)
                    plotted_words.append(word)
        ax = plt.gca()
        legend_without_duplicate_labels(ax)
        #plt.legend(loc=4)"
        # subtitle
        #plt.title(f'\nLabelled with cosine distance of keyword to "{_sentiment}"', fontsize=15, y=1)
        #plt.suptitle(title, fontsize=20)
        plt.title(title + f'\n(Labelled with cosine distance of keyword to "{_sentiment}")')
        plt.ylabel('UMAP 1')
        plt.xlabel('UMAP 2')
        #plt.grid(True)
        plt.tight_layout()
        fig.patch.set_facecolor('#F9F3DC')
        plt.show()

umap_plot_similar_words('UMAP Projection of Word2Vec Model', keys, embeddings_en_2d, word_clusters, 0.8,
                        'similar_words.png')
plt.savefig('umap_similar_words.png')

In [ ]:
# unigram

In [ ]:
model_unigram = Word2Vec.load(str(pathlib.Path('nogit_data', 'Case_1', f'word2vec_unigram_seed-{seed}.model')))

In [ ]:
import matplotlib
import umap.umap_ as umap
def plot_model(model):
    rng = np.random.default_rng(12345)

    embedding_clusters = []
    word_clusters = []
    keys= ['wife', 'worry', 'SO', 'table', 'parking', 'covid', 'LEO', 'MMS', 'herpes']
    # NUMBER IS 10
    colors = ['red', 'blue', 'gold', 'purple', 'pink', 'gray', 'teal', 'orange', 'hotpink']

    SEED = 88
    WORDS = 8

    for word in keys:
        print(f"Key = {word}")
        embeddings = [model.wv[word]]
        words = [word]
        for similar_word, _ in model.wv.most_similar(word, topn=WORDS):
            words.append(similar_word)
            embeddings.append(model.wv[similar_word])
        embedding_clusters.append(embeddings)
        word_clusters.append(words)
        
    embedding_clusters = np.array(embedding_clusters)
    n, m, k = embedding_clusters.shape
    print(n, m, k)
    #tsne_model_en_2d = TSNE(perplexity=15, n_components=2, init='pca', n_iter=3500, random_state=32, n_jobs=-1)
    umap_2d = umap.UMAP(n_neighbors=10, n_components=2, init='spectral', random_state=SEED)
    # his means that low values of n_neighbors will force UMAP to concentrate on very local structure (potentially to the detriment of the big picture), while large values will push UMAP to look at larger neighborhoods of each point wh, default 10

    #umap_3d = UMAP(n_components=3, init='spectral', random_state=0)
    embeddings_en_2d = np.array(umap_2d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 2)
    #embeddings_en_3d = np.array(umap_3d.fit_transform(embedding_clusters.reshape(n * m, k))).reshape(n, m, 3)

    def legend_without_duplicate_labels(ax):
        handles, labels = ax.get_legend_handles_labels()
        unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
        ax.legend(*zip(*unique))

    # hack - why are embeddings a bit different ... 
    def umap_plot_similar_words(title, labels, embedding_clusters, word_clusters, a, filename=None):
        figsize = (9.5,6) if (matplotlib.get_backend() == 'nbAgg') else (12,12)  # interactive plot should be smaller
        #figsize = (9.5,6)
        fig = plt.figure(figsize=(figsize))
        #ax = fig.add_subplot(projection='3d')

        #colors = cm.rainbow(np.linspace(0, 1, len(labels)))
        #colors = matplotlib.colormaps['PiYG'](np.linspace(0, 1, len(labels)))
        #colors = matplotlib.colormaps['Spectral'](np.linspace(0, 1, len(labels)))
        plotted_words = []
        for label, embeddings, words, color in zip(labels, embedding_clusters, word_clusters, colors):
            #x = embeddings[:, 0]
            #y = embeddings[:, 1]
            #plt.scatter(x, y, c=[color], alpha=a, label=label, edgecolors='gray', s=50)
            for i, word in enumerate(words):
                if word not in plotted_words:
                    x = embeddings[i, 0]
                    y = embeddings[i, 1]
                    plt.scatter(x, y, c=[color], alpha=a, label=label, edgecolors='gray')
                    if word =='worry':
                        print(f'{x=}, {y=}')
                    sign = -1 if np.random.random() > 0.5 else 1
                    plt.annotate(word, alpha=.8, xy=(x, y), xytext=(0,0),#(2 + rng.random()*4, sign*(4 + rng.random()*4)),
                                textcoords='offset points', ha='right', va='bottom', size=10)
                    plotted_words.append(word)
        ax = plt.gca()
        legend_without_duplicate_labels(ax)
        #plt.legend(loc=4)"
        plt.title(title)
        plt.ylabel('UMAP 1')
        plt.xlabel('UMAP 2')
        #plt.grid(True)
        plt.show()

    umap_plot_similar_words('UMAP Similar words', keys, embeddings_en_2d, word_clusters, 0.8,
                            'similar_words.png')
    plt.savefig('umap_similar_words.png')
    return fig 


In [ ]:
fig = plot_model(model)
fig

In [ ]:
# against two words

In [ ]:
print(model.wv.similarity('hi', 'hello'))
print(model.wv.similarity('hi', 'table'))

from itertools import permutations

In [ ]:
def get_affinities(feeling, nouns):
    scores = {}
    print()
    for noun in nouns:
        pair = [feeling, noun]
        print('---', ', '.join(pair))
        print('\t\t', f'{model.wv.similarity(*pair):.2f}')
        scores[noun] = f'{model.wv.similarity(*pair):.2f}'
    return scores 

nouns = ['SO', 'wife', 'marriage','friends', 'family', 'LEO', 'police', 'COVID', 'herpes', 'C19', 'arrested', 'kids']
nouns = nonconcern_seeds + concern_seeds + hypothesis_seeds + ['MMS', 'provider']
get_affinities('worry', nouns)
get_affinities('afraid', nouns)
all_feelings = {}
for sentiment in negatives:
    all_feelings[sentiment] = get_affinities(sentiment, nouns)

my_data = pd.DataFrame(all_feelings)

In [ ]:
import altair as alt

In [ ]:
my_data.reset_index()
my_data['color'] = _range
my_data

In [ ]:
my_data.reset_index().sort_values(by='anxiety', ascending=False)

In [ ]:
import altair as alt

In [ ]:
display(alt.__version__ )# must be at least 5.0.0 or else "method based syntax" does not work (for setting title)     
# As described in the release notes for Vega-Lite 5.0.0, t

In [ ]:
for sentiment in negatives: 
    print(sentiment)
    #_sort = my_data[sentiment].sort_values(ascending=False).values
    #print(_sort)
    colors = ['gold', 'gold', 'gray', 'gray', 'purple', 'purple', 'darkgreen', 'darkgreen']
    _domain = nouns 
    _range = colors #+ ['gray', 'black', 'hotpink']
    print(list(zip(_range, _domain)))
    my_data = my_data.sort_values(by=sentiment, ascending=False)
    my_data['order'] = np.arange(0, my_data.shape[0])
    display(my_data)
    my_chart = alt.Chart(my_data.reset_index()).mark_bar().encode(
        x=alt.X('index:N').sort('-y'),
        #x=alt.X('index:N'),
        #x=alt.X('index:N').sort('-y'), #sort=alt.EncodingSortField(field=f'{sentiment}:Q', op='count', order='ascending')),
        #x=alt.X('index:N', sort=_sort), 
        y=alt.Y(f'{sentiment}:Q'),
        #text=f'{sentiment}:Q',
        order='order:Q', 
        color=alt.Color('index').scale(domain=_domain, range=_range)
    ).properties(height=alt.Step(20))
    #display(my_chart.mark_bar() + my_chart.mark_text(align='center', dy=-5))
    #display(my_chart.mark_bar() + my_chart.mark_text())
    display((my_chart + my_chart.mark_text().encode(
        text=f'{sentiment}:Q',
        order='order:Q')).resolve_scale(y='independent'))#.save('test.html')
    display(my_chart)

In [ ]:

testDf = pd.DataFrame({
    'Country/Region': ['China', 'France', 'Italy', 'US'],
    'Color': ['#d62728', '#d62728', '#d62728', '#1f77b4'],
    'Total Deaths': [3180, 1266, 133, 8513],
})

bars = alt.Chart(testDf).mark_bar().encode(
    x = 'Total Deaths',
    y = alt.Y('Country/Region', sort='-x'),
    color = alt.Color('Color', scale = None)
).properties(width = 800, height = 300, title = 'Ten Countries with the Greatest Death Toll')

text = bars.mark_text(
    align = 'left',
    baseline = 'middle',
    dx = 3
).encode(
    text = 'Total Deaths:Q',
    y = alt.Y('Country/Region', sort='-x', axis=None),
)

(bars + text).resolve_scale(y='independent')


In [ ]:
test_df = my_data.reset_index().rename(columns={'index':'word'})
test_df


In [ ]:
import altair as alt
import pandas as pd

testDf = pd.DataFrame({
    'Country/Region': ['parking', 'table', 'leo', 'std'],
    'Color': ['gold', 'gold', 'gray', 'gray'],
    'Total Deaths': [0.13, 0.21, 0.47, 0.61],
})

bars = alt.Chart(testDf).mark_bar().encode(
    y = 'Total Deaths',
    x = alt.X('Country/Region', sort='-y'),
    color = alt.Color('Color', scale = None)
).properties(width = 800, height = 300, title = 'Ten Countries with the Greatest Death Toll')

text = bars.mark_text(
    align = 'left',
    baseline = 'middle',
    dx = 3
).encode(
    text = 'Total Deaths:Q',
    x = alt.X('Country/Region', sort='-y', axis=None),
)

(bars + text).resolve_scale(x='independent')

# Working graph!

In [ ]:
model_unigram.wv.similarity('anxiety', 'parking')

In [ ]:
forum_utils.set_altair_theme()
_range = ['gold', 'gold', 'gray', 'gray', 'purple', 'purple', 'darkgreen', 'darkgreen']
_range = ['goldenrod', 'goldenrod', 'darkslateblue', 'darkslateblue', 'purple', 'purple', 'forestgreen', 'forestgreen']
my_data = pd.DataFrame(all_feelings)
my_data['color'] = _range
#display(my_data)
test_df = my_data.reset_index().rename(columns={'index':'word'})
charts = []
for sentiment in negatives:
    title = alt.TitleParams(
            text= 'Cosine Similarities', 
            subtitle=f'From least to most similar',
            #subtitle=f'Between "{sentiment}" and Keywords',
            align='left',
            dy=-3
        , anchor='start')
    bars = alt.Chart(test_df, title=title).mark_bar(size=20).encode(
        x=alt.X('word:N', sort='y', title='', axis=alt.Axis(labelAngle=-70)), 
        y=alt.Y(f'{sentiment}:Q', title=f'{sentiment.title()}'),
        color = alt.Color(
            'color:N', scale = None
            #alt.Scale(domain=_domain, range=_range) 
        )
    ).properties()
    text = bars.mark_text(align='center', dy=-8,size=14).encode(
        text=f'{sentiment}:Q',
        x=alt.X('word:N', sort='y', axis=None),
    )
    #(bars + text).resolve_scale(y='independent')
    my_chart = (bars+text).resolve_scale(x='independent').properties(
            width=200, height=350
        )
    '''
    .properties( title={ 'text': f'Cosine Similarities', 'subtitle': f'Between "{sentiment}" and Key Words'
            })#.configure_title(anchor='middle')
            
    '''
    charts.append(my_chart)
    display(my_chart)


In [ ]:
#https://github.com/altair-viz/altair/issues/820

#x=alt.X('index:N').sort('-y'), #sort=alt.EncodingSortField(field=f'{sentiment}:Q', op='count', order='ascending')),
# https://github.com/altair-viz/altair/issues/2013
sentiment = 'worry'
_range = ['darkgoldenrod', 'darkgoldenrod', 'gray', 'gray', 'purple', 'purple', 'darkgreen', 'darkgreen']
_domain = nouns 
print(list(zip(_range, _domain)))
sort = alt.EncodingSortField(field='worry', order='descending')

bars = alt.Chart(my_data.reset_index()).mark_bar().encode(
    x=alt.X('index:N', sort='-y'), 
    y=f'{sentiment}:Q', text=f'{sentiment}:Q',
    color=alt.Color('index').scale(domain=_domain, range=_range)
).properties()

text = bars.mark_text().encode(
    x=alt.X('index:N', sort='-y', axis=None),#, axis=alt.Axis(labels=False, ticks=False)),#.sort('-y'),
    text=f'{sentiment}:Q',
)

#display(my_chart.mark_bar()+ my_chart.mark_text(align='center', dy=-5))
#alt.layer(bars, text).resolve_scale(x='independent')
(bars+text).resolve_scale(x='independent')
#(bars+text).save('tmp.html')
#display(my_chart)

In [ ]:
import altair as alt
alt.themes.enable('none')

from vega_datasets import data

sort = alt.EncodingSortField(field='WW_Margin', op='median', 
                             order='descending')

bars = alt.Chart().mark_bar(size=5.0).encode(
    alt.X('Major_Genre:N', sort=sort),
    y='q1(WW_Margin):Q',
    y2='q3(WW_Margin):Q'
)

ticks = alt.Chart().mark_tick(
    color='black',
    size=5.0
).encode(
    alt.X('Major_Genre:N', sort=sort, axis=alt.Axis(labels=False, ticks=False)),
    y='median(WW_Margin):Q',
)

alt.layer(bars, ticks, data=data.movies.url).transform_calculate(
    WW_Margin = "(datum.Worldwide_Gross - datum.Production_Budget) / 1e6"
).resolve_scale(x='independent')

In [ ]:
alt.Chart(my_data.reset_index()).mark_bar().encode(
    x=alt.X('index:N').sort('y'),
    y=alt.Y('afraid:Q')
).properties(height=alt.Step(20))

In [ ]:
alt.Chart(my_data.reset_index()).mark_bar().encode(
    x=alt.X('index:N').sort('y'),
    y=alt.Y('anxious:Q')
).properties(height=alt.Step(20))

In [ ]:
!pip install wordcloud

In [ ]:
import wordcloud

In [ ]:
df['preprocessed_str'] = df.preprocessed_posts.parallel_apply(lambda x: ' '.join(x))

In [ ]:
df.preprocessed_str

In [ ]:
from collections import Counter
from wordcloud import WordCloud
word_cloud_dict=Counter(df.preprocessed_str)
wordcloud = WordCloud(width = 800, height = 800,
                    background_color='white',).generate_from_frequencies(word_cloud_dict)


In [ ]:
plt.figure()
plt.imshow(wordcloud)
plt.axis("off")

In [ ]:
alt.__version__
#!pip install altair==5.0.0rc1

In [ ]:
for pair in pair_list:
    print(pair)